# TIMING II Pipeline Demo
Welcome to TIMING II Pipeline!  This notebook will walk you step by step through the whole TIMING Pipeline, which includes the nanowell detection, cell segmentationa, tracking and feature calculation. The visualization module works independently from another terminal.

# Imports

In [1]:
import sys, time

## Env setup

In [2]:
CORE_NUMBER = 12   #Multi-core speed up setting

TIMING_II_HOME = 'E:\\TIMING\\TIMING2\\'

############################################## Don't Modify below in this code cell ################################### 
# This is needed for nanowell detection.
sys.path.append(TIMING_II_HOME + 'timing2-preprocessing\\')

# This is needed for nanowell detection.
sys.path.append(TIMING_II_HOME + 'timing2-crop\\faster-rcnn\\')

# This is needed for nanowell cropping.
sys.path.append(TIMING_II_HOME + 'timing2-crop\\')

# This is needed for segmentation modules
sys.path.append(TIMING_II_HOME + 'timing2-seg\\')

# This is needed for Cell Tracking.
sys.path.append(TIMING_II_HOME + 'timing2-tracker\\')

# This is needed for Utility functions.
sys.path.append(TIMING_II_HOME + 'timing2-utils\\')

# This is needed for feature calculation modules
sys.path.append(TIMING_II_HOME + 'timing2-features\\')

## TIMING II Parameter Configuration
Set up the parameters below before running the process for a batch of input data.

In [3]:
###### Part 1: Dataset Parameters
Dataset_Name = '20180223_MM_02_ctrl'
Data_Raw_DIR = 'E:\\TIMING\\TIMING2_Datasets_Raw'
Data_DIR = 'E:\\TIMING\\TIMING2_Benchmark_Datasets'

Dataset_Input = 'IN'
Dataset_Output = 'OUT'

Dataset_Blocks = ['B'+str(i).zfill(3) for i in range(1,11)]
Dataset_Channels = ['CH0','CH1','CH2','CH3']                     # Channels in the experiment, don't need to be changed
Dataset_Frames = 73
Block_Size = 2048 #pixels
Nanowell_Size = 281 #pixels

### Part 2: Preprocessing Parameters
stack_tuple = ("bright_field","effectors","targets","death")     # channels that are stacked together

unmix_tuple = ("targets","effectors", "death") # channels that are used in unmixing  
unmix_tuple_clean = ("targets", "effectors")   # if channels in this tuple, preprocessor will look for umx_Bxxx
unmix_ratio_c2_c3 = 0.77                        # otherwise, will use Bxxx to generate new bg_Bxxx    
unmix_ratio_c3_c4 = 0.22

enhance_tuple = ("targets",)
min_pixel_value = 0
max_pixel_value = 2000

preprocess_tuple = ("effectors","targets")      # channels that have umx and back ground subtraction

channel_dict = {"bright_field":"c1_ORG",
                "effectors":"c4_ORG",
                "targets":"c3_ORG",
                "death":"c2_ORG",
                "beads_1":"--",
                "beads_2":"--"}

channel_naming_dict = {"bright_field":"0",
                       "effectors":"1",
                       "targets":"2",
                       "death":"3",
                       "beads_1":"4",
                       "beads_2":"5"}
microscope = 'zeiss'


### Part 3: Pipeline Parameters

# Channels Available
Channel_available = ['CH0', 'CH1', 'CH2', 'CH3']      # Available Channels in experiments

# Segmentation Radius for Channel 1 and 2
R_CH1 = 20   # 23
R_CH2 = 21   # 24

# Resegmentation Configuration
CELL_COUNT_THRESHOLD = 0.8
CELL_COUNT_MAX = 4

# TIMING II Pipeline Step by Step

## A. Preprocessing


Step 0: Make directory for preprocessing, only need to run one time

In [4]:
from Preprocess_Wrapper import TIMING_Preprocess_Mkdir

TIMING_Preprocess_Mkdir(Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks)

In [5]:
### Put all the Preprocessing in a block by block cycle
from Preprocess_Wrapper import TIMING_Preprocess_Stack
from Preprocess_Wrapper import TIMING_Preprocess_Unmix
from Preprocess_Wrapper import TIMING_Preprocess_BackgroundSubtract
from Preprocess_Wrapper import TIMING_Preprocess_Enhance


for Block in Dataset_Blocks:
        t1 = time.time()
        
        print("Start Pre-Processing " + Block + " ......")
        
        Blocks = [Block]
        
        # Stack the Images
        TIMING_Preprocess_Stack(TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, stack_tuple, channel_dict, channel_naming_dict, microscope, CORE_NUMBER)

        # Unmixing c2_ORG and c3_ORG
        TIMING_Preprocess_Unmix(2, 0, unmix_ratio_c2_c3, TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, unmix_tuple, stack_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)
        
        #Unmixing c3_ORG and c4_ORG
        TIMING_Preprocess_Unmix(0, 1, unmix_ratio_c3_c4, TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, unmix_tuple, stack_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)
        
        #Background Subtraction
        TIMING_Preprocess_BackgroundSubtract(40, TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, preprocess_tuple, stack_tuple, unmix_tuple_clean, channel_dict, channel_naming_dict, CORE_NUMBER)
        
        #Channel Enhancement
        TIMING_Preprocess_Enhance(min_pixel_value, max_pixel_value,TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Blocks, enhance_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)
        
        print("Preprocessing " + Block +" TIME: " +str(time.time() - t1))



Start Pre-Processing B001 ......
Preprocessing B001 TIME: 156.09792828559875
Start Pre-Processing B002 ......
Preprocessing B002 TIME: 193.8412435054779
Start Pre-Processing B003 ......
Preprocessing B003 TIME: 173.13560438156128
Start Pre-Processing B004 ......
Preprocessing B004 TIME: 159.08847975730896
Start Pre-Processing B005 ......
Preprocessing B005 TIME: 159.84338092803955
Start Pre-Processing B006 ......
Preprocessing B006 TIME: 174.72680711746216
Start Pre-Processing B007 ......
Preprocessing B007 TIME: 171.63900184631348
Start Pre-Processing B008 ......
Preprocessing B008 TIME: 142.02814960479736
Start Pre-Processing B009 ......
Preprocessing B009 TIME: 139.22014474868774
Start Pre-Processing B010 ......
Preprocessing B010 TIME: 164.33618903160095


Step 1: Stack the Images

In [ ]:
# from Preprocess_Wrapper import TIMING_Preprocess_Stack

# t1 = time.time()

# TIMING_Preprocess_Stack(TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, stack_tuple, channel_dict, channel_naming_dict, microscope, CORE_NUMBER)

# print("1-STACK TIME: " +str(time.time() - t1))

Step 2.1: Unmixing the images, remove c2_ORG from c3_ORG, leaked from unmix_tuple[2] to unmix_tuple[0] 

In [ ]:
# from Preprocess_Wrapper import TIMING_Preprocess_Unmix

# t1 = time.time()

# TIMING_Preprocess_Unmix(2, 0, unmix_ratio_c2_c3, TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, unmix_tuple, stack_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)

# print("2.1-UNMIX TIME: " +str(time.time() - t1))

Step 2.2: Unmixing the images, remove c3_ORG from c4_ORG

In [ ]:
# from Preprocess_Wrapper import TIMING_Preprocess_Unmix

# t1 = time.time()

# TIMING_Preprocess_Unmix(0, 1, unmix_ratio_c3_c4, TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, unmix_tuple, stack_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)

# print("2.2-UNMIX TIME: " +str(time.time() - t1))

Step 3: Background Subtraction

In [ ]:
# from Preprocess_Wrapper import TIMING_Preprocess_BackgroundSubtract

# t1 = time.time()

# TIMING_Preprocess_BackgroundSubtract(40, TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, preprocess_tuple, stack_tuple, unmix_tuple_clean, channel_dict, channel_naming_dict, CORE_NUMBER)

# print("3-BS TIME: " +str(time.time() - t1))

Step 4: Channel Enhancement because Target Channel Pixel value too small

In [ ]:
# from Preprocess_Wrapper import TIMING_Preprocess_Enhance

# t1 = time.time()

# TIMING_Preprocess_Enhance(min_pixel_value, max_pixel_value,TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, enhance_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)

# print("4-Channel Enhancement TIME: " +str(time.time() - t1))            


Step 5: Do a possible clip Enhancement to Effector Channels if pixel value too noise std > 600

In [ ]:
# min_clip_value = 100
# max_clip_value = 1000

# clip_enhance_tuple = ("Effectors",)

# from Preprocess_Wrapper2 import TIMING_Preprocess_Clip_Enhance

# t1 = time.time()

# TIMING_Preprocess_Enhance(min_clip_value, max_clip_value,min_pixel_value, max_pixel_value,TIMING_II_HOME, Data_Raw_DIR, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, clip_enhance_tuple, channel_dict, channel_naming_dict, CORE_NUMBER)

# print("5-Channel Clip-Enhancement TIME: " +str(time.time() - t1))      

## B. Nanowell Localization, Classification and Cropping
 We use faster r-cnn to do nanowell localization and identify the empty ones which we will discard automatically. Small image patch will be cropped right after the localization.

In [6]:
from Crop_Wrapper import TIMING_Crop

t1 = time.time()

TIMING_Crop(TIMING_II_HOME, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, Dataset_Frames, Nanowell_Size, Block_Size, Channel_available)

print("STEP-CROPPING TIME: " +str(time.time() - t1))

C:\Users\mamart51\Anaconda2\envs\TIMING2-pipeline\lib\site-packages\skimage\util\dtype.py:122: UserWarning: Possible precision loss when converting from uint16 to uint8
  .format(dtypeobj_in, dtypeobj_out))


processing B001 ......
processing B002 ......
processing B003 ......
processing B004 ......
processing B005 ......
processing B006 ......
processing B007 ......
processing B008 ......
processing B009 ......
processing B010 ......
Processing B001 ......
Processing B002 ......
Processing B003 ......
Processing B004 ......
Processing B005 ......
Processing B006 ......
Processing B007 ......
Processing B008 ......
Processing B009 ......
Processing B010 ......
STEP-CROPPING TIME: 1477.6816055774689


## C. Cell Segmentation 
We use i-vote to detect the seeds for cell candidates. And we use k-means to extract the foreground of cells from the fluorescent channels(CH1 and CH2). Individual cell bodies are clustered based on their distance from each detected seeds.

In [7]:
 from Segment_Wrapper import TIMING_Segment

t1 = time.time()

# TIMING_Segment(TIMING_II_HOME, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER, Nanowell_Size, Block_Size, Channel_available, R_CH1, R_CH2)

Thresh_CH1= 0.4    # Needs to specify this threshold value for two channels, the lower the value, the more cell bodies

Thresh_CH2= 0.2

TIMING_Segment(TIMING_II_HOME, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER, Nanowell_Size, Block_Size, Channel_available, Thresh_CH1, Thresh_CH2, R_CH1, R_CH2)

print("STEP-CELL SEGMENTATION TIME: " +str(time.time() - t1))

Processing B001 ......
Processing B002 ......
Processing B003 ......
Processing B004 ......
Processing B005 ......
Processing B006 ......
Processing B007 ......
Processing B008 ......
Processing B009 ......
Processing B010 ......
Processing B001 ......
Processing B002 ......
Processing B003 ......
Processing B004 ......
Processing B005 ......
Processing B006 ......
Processing B007 ......
Processing B008 ......
Processing B009 ......
Processing B010 ......
STEP-CELL SEGMENTATION TIME: 1485.4080126285553


## D. Confinement-Constrained Cell Resegmentation
Assume the numbers of cells detected are constant throughout the whole time line. 

In [8]:
# from Segment_Wrapper2 import TIMING_Segment2
9
# t1 = time.time()

# TIMING_Segment2(TIMING_II_HOME, Data_DIR, Dataset_Name, Dataset_Input, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER, Nanowell_Size, Block_Size, R_CH1, R_CH2)

# print("STEP-CELL SEGMENTATION TIME: " +str(time.time() - t1))

9

In [9]:
from Resegment_Wrapper import TIMING_Resegment

t1 = time.time()

TIMING_Resegment(Data_DIR, Dataset_Name, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER, CELL_COUNT_THRESHOLD, CELL_COUNT_MAX)

print("STEP-CELL RE-SEGMENTATION TIME: " +str(time.time() - t1)) 
        

STEP-CELL RE-SEGMENTATION TIME: 74.13133001327515


## E. Cell Tracking
Based on the results from cell segmentation. We define the cost functions for cell track assignment with cell center distance measure, cell size difference measure and set distance measure. We try to find the optimal tracking result by minimizing the cost function.

In [10]:
from Tracking_Wrapper2 import TIMING_Tracker2

t1 = time.time()

TIMING_Tracker2(Data_DIR, Dataset_Name, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER)

print("STEP-CELL TRACKING TIME: " +str(time.time() - t1))

STEP-CELL TRACKING TIME: 10.655818700790405


## F. Cell Feature Calculation
We quantify the cells in each identified tracks. For Effector cells in Channel 1, we calculate the centroid position, aspect ratio, speed and death marker level at each time point. For Target cells in Channel 2, we calculate an additional contact ratio.

In [11]:
from Feature_Wrapper import TIMING_Features

t1 = time.time()

TIMING_Features(Data_DIR, Dataset_Name, Dataset_Output, Dataset_Blocks, Dataset_Frames, CORE_NUMBER)

print("STEP-FEATURE CALCULATION TIME: " +str(time.time() - t1))

......
STEP-FEATURE CALCULATION TIME: 87.8940544128418
